# Integrate Amazon SageMaker with DataRobot MLOps

Authors: Oleksandr Saienko, DataRobot
Mao Shun, AWS

Version 1.3 (04/03/2022)

With Amazon SageMaker, you can package your own algorithms that can than be trained and deployed in the SageMaker environment. [DataRobot MLOps](https://docs.datarobot.com/en/docs/mlops/index.html) monitoring provides service health, data drift, accuracy monitoring, reports, and alerts about machine learning performance. This notebook is modified based on a [SageMaker example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb) to show integration capabilities of DataRobot MLOps. 

To integrate with SageMaker, you must first build and register a SageMaker container.

The README demonstrates how to build a custom SageMaker container in your local environment by including custom Python libraries for both training and inference. The README also includes DataRobot-related libraries useful for model monitoring.

Additionally, modify the Dockerfile as you need and follow the command instructions.

Once you have your container packaged, you can use it to train models and use the model for hosting.

DataRobot recommends running the cells below in a SageMaker notebook instance for simplicity. If you want to run it locally, some settings need be added.

## Setup

Specify a bucket to use and the role used when working with SageMaker.

In [67]:
# S3 prefix
prefix = "DEMO-scikit-byo-iris-v3"

# Define IAM role
import boto3
import re

import json
import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

arn:aws:iam::293058073847:role/service-role/AmazonSageMaker-ExecutionRole-20220606T111248


### Create a session

The session remembers your connection parameters to SageMaker. Use it to perform all of the SageMaker operations.

In [68]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

### Import data

This example workflow uses the [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) and is included in the notebook folder.

Use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [69]:
WORK_DIRECTORY = "data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit your algorithm, create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed in the shell commands above.
* The __role__. Defined above.
* The __instance count__ is the number of machines to use for training.
* The __instance type__ is the type of machine to use for training.
* The __output path__ determines where the model artifact is written.
* The __session__ is the SageMaker session object that you defined above.

Use fit() on the estimator to train against the data uploaded above.

In [70]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-datarobot-decision-trees:latest".format(account, region)

print(image)

print("data_location")
print(data_location)

tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)

tree.fit(data_location)

293058073847.dkr.ecr.us-east-1.amazonaws.com/sagemaker-datarobot-decision-trees:latest
data_location
s3://sagemaker-us-east-1-293058073847/DEMO-scikit-byo-iris-v3
2023-03-23 10:54:00 Starting - Starting the training job...
2023-03-23 10:54:24 Starting - Preparing the instances for trainingProfilerReport-1679568840: InProgress
......
2023-03-23 10:55:24 Downloading - Downloading input data..Starting the training.
Training complete.

2023-03-23 10:55:52 Training - Training image download completed. Training in progress.
2023-03-23 10:55:52 Uploading - Uploading generated training model
2023-03-23 10:55:52 Completed - Training job completed
Training seconds: 37
Billable seconds: 37


## Configure DataRobot MLOps

Before proceeding with the workflow, install a pip package in the current kernel.

In [71]:
import sys
#installing DataRobot MLOps client
!{sys.executable} -m pip install datarobot-mlops

#installing mlops-cli tool
!{sys.executable} -m pip install datarobot-mlops-connected-client

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


### Connect to DataRobot

To use the DataRobot API, you first need to [create an API key](https://docs.datarobot.com/en/docs/api/api-quickstart/index.html#create-a-datarobot-api-key).

Then, add `MLOPS_SERVICE_URL` and `MLOPS_API_TOKEN` as environment variables.

In [99]:
%env MLOPS_SERVICE_URL=https://app.datarobot.com
#PUT Your DataRobot API Key here:
%env MLOPS_API_TOKEN=PUT_YOUR_API_TOKEN

env: MLOPS_SERVICE_URL=https://app.datarobot.com
env: MLOPS_API_TOKEN=YOUR_API_TOKEN


### Upload a training dataset to DataRobot AI Catalog

In the UI, you can [import a dataset via the AI catalog](https://app.datarobot.com/docs/data/ai-catalog/catalog.html#add-data).

Alternatively, you can use `mlops-cli` from the command line as demonstrated in the cells below.

In [76]:
%%capture cap --no-stderr
# ^^^^ Just to catch mlops-cli commands output to process it programmatically, comment it for cell output 
# Load the training dataset using mlops-cli, 
# we are using --json --quiet options here to catch command output as a json to process it programmatically
# if you need text output you can use --terse option
!mlops-cli dataset upload --input "data/iris_with_header.csv" --timeout 600 --json --quiet

In [77]:
# Output of this command will contain uploaded Dataset ID in 'id' field that needs to be used on the next steps:
print(cap.stdout)
if "ERROR" not in cap.stdout: 
    stdout_json = json.loads(cap.stdout)
    print("Training dataset uploaded successfully, TRAINING_DATASET_ID="+stdout_json['id'])
    #Setting TRAINING_DATASET_ID env variable to use it in the next steps:
    os.environ["TRAINING_DATASET_ID"] = stdout_json['id']
else:
    # Print output of mlops-cli in case of error:
    print("Training dataset uploading failed:")
    print(cap.stdout) 

{
 "id": "641c32d631185440a581d435"
}

Training dataset uploaded successfully, TRAINING_DATASET_ID=641c32d631185440a581d435


## Create a model package

In the UI, you can view existing model packages or add a new one by navigating to [**Model Registry > Model Packages**](https://app.datarobot.com/docs/mlops/deployment/registry/reg-create.html#create-model-packages).

Alternatively, you can use `mlops-cli` as shown in the following cells:

In [78]:
MODEL_PACKAGE_NAME="SageMaker_MLOps_Demo_v2"

#Set model type
prediction_type="Multiclass"
#Set traget column
model_target = "variety"
#Set traget classes
class_names = ["setosa", "versicolor", "virginica"]

model_config = {
    "name": MODEL_PACKAGE_NAME,
    "modelDescription": {
    "modelName": "Iris classification model",
    "description": "Classification on iris dataset"
    },
    "target": {
        "type": prediction_type,
        "name": model_target,
        "classNames": class_names
    }
}

# write model configuration json to a file:
with open("demo_model.json", "w") as model_json_file:
    model_json_file.write(json.dumps(model_config, indent=4))

In [79]:
%%capture cap --no-stderr
# Create model package
# we are using --json --quiet options here to catch command output as a json to process it programmatically
# if you need text output you can use --terse option
# Using Dataset ID from previouse step as a training-dataset-id argument:
!mlops-cli model create --json-config "demo_model.json" --training-dataset-id $TRAINING_DATASET_ID  --json --quiet
# Output of this command will contain json with created model package ID that needs to be used on the next steps:

In [80]:
print(cap.stdout) #Just to check mlops-cli command output
if "ERROR" not in cap.stdout:
    #catch Model Package ID corresponding variable:
    stdout_json = json.loads(cap.stdout)
    print("Model package created successfully, MODEL_PACKAGE_ID="+stdout_json['id'])
    #Setting TRAINING_DATASET_ID env variable to use it in the next steps:
    os.environ["MODEL_PACKAGE_ID"] = stdout_json['id']
    #set Model Package ID corresponding variable:
    model_id=stdout_json['id']
else:
    # Handle or output of mlops-cli in case of error:
    print("Model package creation failed:")
    print(cap.stdout) 

{
 "id": "641c332175cd8d976294d58b"
}

Model package created successfully, MODEL_PACKAGE_ID=641c332175cd8d976294d58b


## Create a DataRobot prediction environment

Models that run on your own infrastructure (outside of DataRobot) may be run in different environments and can have differing deployment permissions and approval processes. 
To deploy models on external infrastructure, you need create a custom external prediction environment using the UI or the DataRobot API and copying the prediction environment ID.

For more information, reference the documentation for [creating external prediction environments](https://app.datarobot.com/docs/mlops/deployment/ext-model-prep/pred-env.html).

To create a prediction environment from `mlops-cli` you can use the following cells:

In [81]:
# Create a  configuration
demo_pe_config = {
    "name": "MLOps SageMaker Demo v2",
    "description": "AWS Sagemaker DataRobot MLOps Demo",
    "platform": "aws",
    "supportedModelFormats": ["externalModel"]
}

# Write the configuration json to a file
with open("demo_pe.json", "w") as demo_pe_file:
    demo_pe_file.write(json.dumps(demo_pe_config, indent=4))

In [82]:
%%capture cap --no-stderr 
# Used to catch mlops-cli commands output
# Run this only once, or at least clean up after so you don't end up with a lot of deployments
!mlops-cli prediction-environment create --json-config "demo_pe.json"  --json --quiet
# The output of this command will contain a prediction environment ID that is required in the following cells

In [85]:
# The output of this command contains a prediction environment ID that is required in the following cells
print(cap.stdout) #Just to check mlops-cli command output
if "ERROR" not in cap.stdout:
    # Used to catch prediction environment corresponding variable:
    stdout_json = json.loads(cap.stdout)
    print("Prediction environment created successfully, PREDICTION_ENVIRONMENT_ID="+stdout_json['id'])
    # Set the PREDICTION_ENVIRONMENT_ID environment variable to use it in the following cells
    os.environ["PREDICTION_ENVIRONMENT_ID"] = stdout_json['id']
else:
    # Handle or output of mlops-cli in case of error:
    print("Prediction environment creation failed:")
    print(cap.stdout) 


{
 "id": "641c337702b02e65c447d3b0"
}

Prediction environment created successfully, PREDICTION_ENVIRONMENT_ID=641c337702b02e65c447d3b0


In [86]:
%%capture cap --no-stderr 
# In the UI, you can create a deployment from a model package under Model Registry -> {model package} -> Deployments.
# Set --deployment-label with name that you choose
# --model-package-id from previous step
# --prediction-environment-id from previous step
!mlops-cli model deploy --model-package-id $MODEL_PACKAGE_ID --prediction-environment-id $PREDICTION_ENVIRONMENT_ID --deployment-label "SageMaker_MLOps_Demo"  --json --quiet

In [88]:
# The output of this command contains a deployment ID that is required in the following cells
print(cap.stdout) # Used to check mlops-cli command output
if "ERROR" not in cap.stdout:
    # Used to catch deployment ID corresponding variable:
    stdout_json = json.loads(cap.stdout)
    print("Model deployment created successfully, DEPLOYMENT_ID="+stdout_json['id'])
    # Set the deployment_id variable to use it in the next steps:
    deployment_id = stdout_json['id']
else:
    # Handle or output of mlops-cli in case of error:
    print("Model deployment creation failed:")
    print(cap.stdout) 

{
 "id": "641c345ccf103feaf4d2e1a4"
}

Model deployment created successfully, DEPLOYMENT_ID=641c345ccf103feaf4d2e1a4


## Create an SQS queue as a spooler channel

The MLOps library communicates to the MLOps agent using a spooler. This workflow uses AWS SQS as a spooler channel, more details how to create SQS queue:
You can read more about how to [create an SQS queue using the cloud console UI](https://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/step-create-queue.html) or [by using the AWS CLI](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/sqs/create-queue.html).

In [89]:
#!aws sqs create-queue --queue-name datarobot-mlops-demo-v2
# MLOps spooler channel SQS queue
# Put your SQS queue URL here:
MLOPS_SQS_QUEUE="https://sqs.us-east-1.amazonaws.com/12345678/aws-mlops-blogpost-demo"

## Host your model

You can use a trained model to get real time predictions using an HTTP endpoint.

### Deploy the model

Deploying a model to SageMaker hosting just requires a `deploy` call with the fitted model. This call requires an instance count, instance type, and optional serializer and deserializer functions. These functions are used when the resulting predictor is created on the endpoint.

In [90]:
from sagemaker.serializers import CSVSerializer
import json

# Pass all required environment variables to the SageMaker deployment
env_vars={
    "MLOPS_DEPLOYMENT_ID": deployment_id,
    "MLOPS_MODEL_ID": model_id,
    "MLOPS_SQS_QUEUE": MLOPS_SQS_QUEUE,
    "prediction_type": prediction_type,
    "CLASS_NAMES": json.dumps(class_names)}

print(env_vars)

predictor = tree.deploy(1, "ml.m4.xlarge", serializer = CSVSerializer(), env=env_vars)

{'MLOPS_DEPLOYMENT_ID': '641c345ccf103feaf4d2e1a4', 'MLOPS_MODEL_ID': '641c332175cd8d976294d58b', 'MLOPS_SQS_QUEUE': 'https://sqs.us-east-1.amazonaws.com/293058073847/aws-mlops-blogpost-demo', 'prediction_type': 'Multiclass', 'CLASS_NAMES': '["setosa", "versicolor", "virginica"]'}
-------!

### Get prediction data

In order to make predictions, extract the data used for training to make predictions against it. This is is strictly for demo purposes as it is bad statistical practice. However it is a good demonstration of how the mechanism works.

In [91]:
shape = pd.read_csv("data/iris_with_header.csv", header=None)
shape.sample(3)

,0,1,2,3,4
65,versicolor,5.6,2.9,3.6,1.3
8,setosa,5,3.4,1.5,0.2
68,versicolor,5.8,2.7,4.1,1


In [92]:
# Drop the label column in the training set
shape.drop(shape.columns[[0]], axis=1, inplace=True)
shape.sample(3)

,1,2,3,4
125,6.7,3.3,5.7,2.1
86,6,3.4,4.5,1.6
29,5.2,3.4,1.4,0.2


In [93]:
import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = shape.iloc[indices[:-1]]

test_data

,1,2,3,4
40,5.1,3.4,1.5,0.2
41,5,3.5,1.3,0.3
42,4.5,2.3,1.3,0.3
43,4.4,3.2,1.3,0.2
44,5,3.5,1.6,0.6
45,5.1,3.8,1.9,0.4
46,4.8,3,1.4,0.3
47,5.1,3.8,1.6,0.2
48,4.6,3.2,1.4,0.2
49,5.3,3.7,1.5,0.2


Making prediction is as easy as calling `predict` with the predictor you get back from `deploy` and the data you want to make predictions with. The serializers convert the data for you.

In [96]:
print(predictor.predict(test_data.values).decode("utf-8"))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



### Cleanup

Optional. When you're done with the endpoint, you can clean it up.

In [39]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
